# Atmospherically Corrected Earth EngineTime Series 

### Overview

This notebook creates an atmospherically corrected Sentinel 2 time series.

### Output

Mean average (cloud-free) pixel values.

#### Initialize

In [ ]:
import os
import sys
import datetime
import numpy as np
from matplotlib import pylab as plt
%matplotlib inline
import ee
ee.Initialize()
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'atmcorr'))
import interpolated_lookup_tables as iLUT
from ee_requests import request_meanRadiance
from atmcorr_timeseries import surface_reflectance_timeseries

#### User Input

In [ ]:
# region of interest
geom = ee.Geometry.Point(-67.6929, -23.4979)

# start and end of time series
startDate = ee.Date('2000-01-01')
stopDate  = ee.Date('2020-01-01')

#### Interpolated look-up tables

The emulator uses interpolated look-up tables to generate atmospheric correction coeffients.

In [ ]:
iLUTs = iLUT.handler() 
iLUTs.mission = 'Sentinel2'
iLUTs.get()

<u>pro tip</u>: you can point to existing interpolated look-up table files (.ilut) for faster execution.

`iLUTs.path = 'path/to/ilut/files'`

#### Image Collection

In [ ]:
ic = ee.ImageCollection('COPERNICUS/S2')\
  .filterBounds(geom)\
  .filterDate(startDate, stopDate)\
  .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',75))

#### Mean Average Radiance

In [ ]:
# earth engine request
request = request_meanRadiance(ic, geom, cloudMask = False)

# get data locally (i.e. client side)
print('Getting data from Earth Engine.. ')
meanRadiance = request.getInfo()
print('Data collection complete')

#### Atmospheric Correction

In [ ]:
timeSeries = surface_reflectance_timeseries(meanRadiance, iLUTs)

### Plots

#### Visible wavelengths

In [ ]:
dates = [datetime.datetime.utcfromtimestamp(dt) for dt in timeSeries['timeStamp']]
plt.plot(dates, timeSeries['B2'], 'blue')
plt.plot(dates, timeSeries['B3'], 'green')
plt.plot(dates, timeSeries['B4'], 'red')
plt.title('RGB')
plt.show()

#### Near infrared

In [ ]:
plt.plot(dates, timeSeries['B5'], '#cbc9e2', label='B5: 705 nm')
plt.plot(dates, timeSeries['B6'], '#9e9ac8', label='B6: 740 nm')
plt.plot(dates, timeSeries['B7'], '#756bb1', label='B7: 783 nm')
plt.plot(dates, timeSeries['B8'], '#54278f', label='B8: 842 nm')
plt.title('Near Infrared')
plt.legend()
plt.show()

#### Short-wave Infrared

In [ ]:
plt.plot(dates, timeSeries['B11'], '#993404', label='B11: 1610 nm')
plt.plot(dates, timeSeries['B12'], '#fe9929', label='B12: 2190 nm')
plt.title('Short-wave Infrared')
plt.legend()
plt.show()

#### Atmospheric bands

In [ ]:
plt.plot(dates, timeSeries['B1'], '#c994c7', label='Aerosol (B1: 443 nm)')
plt.plot(dates, timeSeries['B9'], '#225ea8', label='Water Vapour (B9: 940 nm)')
plt.plot(dates, timeSeries['B10'], '#c7e9b4', label='Cirrus (B10: 1375 nm)')
plt.title('Atmospheric Wavebands')
plt.legend()
plt.show()